# Capítulo 6: Validación

Se realizan chequeos rápidos de calidad: valores faltantes, rangos y distribución de la variable, top/bottom departamentos, y (si aplica) comparación entre métodos de clasificación para detectar outliers o resultados inverosímiles.

**Objetivo:** asegurar que los resultados sean consistentes y reproducibles.  
**Entradas:** unión geográfica y variable objetivo.  
**Salidas:** tablas/resúmenes de validación y observaciones de calidad.


In [ ]:

import pandas as pd
import geopandas as gpd
from pathlib import Path

# 1) Cargar unión del Cap. 4
RUTA_GPKG = Path("..")/"data"/"union_departamentos.gpkg"
gdf = gpd.read_file(RUTA_GPKG, layer="union").to_crs(4326)
print("Filas:", len(gdf), "| Columnas:", list(gdf.columns))

# 2) Elige la variable que mapeaste en Cap. 5
VARIABLE = "DIRECTORIO"        # o "CUENTAS" si usaste conteo
assert VARIABLE in gdf.columns, f"No existe {VARIABLE} en la capa."

# 3) Faltantes por departamento (tabla para el informe)
faltantes = (gdf[gdf[VARIABLE].isna()][["DPTO_CCDGO","DPTO_CNMBR"]]
             .sort_values("DPTO_CCDGO"))
faltantes.reset_index(drop=True, inplace=True)
print(f"Faltantes en {VARIABLE}: {len(faltantes)}")
faltantes

# 4) Estadísticas descriptivas
desc = gdf[VARIABLE].describe()
desc

# 5) Duplicados y cobertura
dup = gdf["DPTO_CCDGO"].duplicated().sum()
print("Duplicados de DPTO_CCDGO:", dup)
cobertura = gdf[VARIABLE].notna().mean()
print(f"Cobertura de la variable: {cobertura:.0%}")

# 6) Exportar validación
out_dir = Path("..")/"data"
out_dir.mkdir(exist_ok=True)
(faltantes.assign(variable=VARIABLE)
         .to_csv(out_dir/f"validacion_faltantes_{VARIABLE}.csv", index=False))
desc.to_frame("valor").to_csv(out_dir/f"validacion_descriptivos_{VARIABLE}.csv")
print("Guardado CSVs de validación en ../data/")


Filas: 33 | Columnas: ['DPTO_CCDGO', 'DPTO_CNMBR', 'DIRECTORIO', 'geometry']
Faltantes en DIRECTORIO: 2
Duplicados de DPTO_CCDGO: 0
Cobertura de la variable: 94%
Guardado CSVs de validación en ../data/


## Validación del dataset geoespacial

Después de la unión entre el shapefile de departamentos y el dataset estadístico, se realizó un proceso de validación de la información.

### Resultados
- **Filas totales:** 33 (corresponde al número de departamentos de Colombia).  
- **Columnas disponibles:** `['DPTO_CCDGO', 'DPTO_CNMBR', 'DIRECTORIO', 'geometry']`.  
- **Valores faltantes en `DIRECTORIO`:** 2 registros sin dato asociado.  
- **Duplicados en `DPTO_CCDGO`:** 0 (cada departamento aparece una sola vez).  
- **Cobertura de la variable:** 94% (la mayoría de los departamentos tienen valores, pero algunos están incompletos).  
- **Archivos de validación:** CSVs exportados en la carpeta `../data/`.  

### Observaciones
- La cobertura del 94% indica que **casi todos los departamentos tienen información válida**, pero un pequeño porcentaje no fue asociado correctamente.  
- La ausencia de duplicados en `DPTO_CCDGO` garantiza que no hay inconsistencias en la clave de unión.  
- Los 2 valores faltantes en `DIRECTORIO` deben ser revisados para decidir si se imputan, corrigen o eliminan.  

### Conclusión
El proceso de validación muestra que el dataset está **limpio y consistente en un 94%**, listo para análisis geoespacial y estadístico. Solo se requiere revisar los registros faltantes en `DIRECTORIO` para alcanzar una cobertura completa.
